# Implementing a Key-Value Database

In this project, we'll implement a key-value database that will be flexible and easy for other developers to use in their own projects. We'll build off of a B-Tree class and turn it into a fully functional key-value store.

A key-value database operates similarly to a Python dictionary, but it allows the users to perform a range of queries. We'll be building a database similar to other open-source implementations of a key-value store like Redis, CouchDB, Mongo, and Cassandra. 

## Implementing a B-Tree

Before we create our key-value database, we'll need to add in a B-tree class implementation.

### Node Class

To implement a B-tree from scratch, we'll need to create a `Node` class so that we can use two separate lists to represent the keys and the children.

We'll also implement an `is_leaf` method so we know if the node is a leaf – has no children – or not. We'll also add a `__repr__()` method so that we can track the number of keys contained within a B-tree node.

In [1]:
class Node:
    
    def __init__(self, keys=None, children=None):
        self.keys = keys or []
        self.children = children or []

    def is_leaf(self):
        return len(self.children) == 0

    def __repr__(self):
        return "<Node: {}>".format(self.keys)

### B-Tree Class

The following code represents the B-tree class.

In [7]:
class BTree:
    def __init__(self, t):
        self.t = t
        self.root = None

    def insert_multiple(self, keys):
        for key in keys:
            self.insert(key)

    def insert(self, key):
        if not self.root:
            self.root = Node(keys=[key])
            return

        if len(self.root.keys) == 2*self.t - 1:
            old_root = self.root
            self.root = Node()
            left, right, new_key = self.split(old_root)
            self.root.keys.append(new_key)
            self.root.children.append(left)
            self.root.children.append(right)

        self.insert_non_full(self.root, key)

    def insert_non_full(self, node, key):
        if node.is_leaf():
            index = 0
            for k in node.keys:
                if key > k: index += 1
                else: break
            node.keys.insert(index, key)
            return

        index = 0
        for k in node.keys:
            if key > k: index += 1
            else: break
        if len(node.children[index].keys) == 2*self.t - 1:
            left_node, right_node, new_key = self.split(node.children[index])
            node.keys.insert(index, new_key)
            node.children[index] = left_node
            node.children.insert(index+1, right_node)
            if key > new_key:
                index += 1

        self.insert_non_full(node.children[index], key)

    def split(self, node):
        left_node = Node(
            keys=node.keys[:len(node.keys)//2],
            children=node.children[:len(node.children)//2+1]
        )
        right_node = Node(
            keys=node.keys[len(node.keys)//2:],
            children=node.children[len(node.children)//2:]
        )
        key = right_node.keys.pop(0)
        return left_node, right_node, key

    def search(self, node, term):
        if not self.root:
            return None
        index = 0
        for key in node.keys:
            if key == term:
                return key.value
            if term > key:
                index += 1
        if node.is_leaf():
            return None
        return self.search(node.children[index], term)

    def greater_than(self, node, term, upper_bound=None, inclusive=False):
        if not self.root:
            return []
        index = 0
        values = []
        for key in node.keys:
            if upper_bound is not None:
                if inclusive and key == upper_bound:
                    values.append(key)
                if key >= upper_bound:
                    break
            if term > key:
                index += 1
                continue
            if inclusive and key == term:
                values.append(key)
            if key > term:
                values.append(key)
            if not node.is_leaf():
                values += self.greater_than(
                    node.children[index],
                    term,
                    upper_bound,
                    inclusive
                )
            index += 1
        if not node.is_leaf():
            values += self.greater_than(
                node.children[index],
                term,
                upper_bound,
                inclusive
            )
        return values

    def less_than(self, node, term, lower_bound=None, inclusive=False):
        if not self.root:
            return []
        index = 0
        values = []
        for key in node.keys:
            if lower_bound is not None:
                if inclusive and key == lower_bound:
                    values.append(key)
                if key < lower_bound:
                    index += 1
                    continue
            if inclusive and key == term:
                values.append(key)
            if key < term:
                values.append(key)
            if not node.is_leaf():
                values += self.less_than(
                    node.children[index],
                    term,
                    lower_bound,
                    inclusive
                )
            index += 1

        if not node.is_leaf() and key <= term:
            values += self.less_than(
                node.children[index],
                term,
                lower_bound,
                inclusive
            )
        return values


class NodeKey:
    def __init__(self, key, value):
        self.key = key
        self.value = value

    def __repr__(self):
        return '<NodeKey: ({}, {})>'.format(self.key, self.value)

    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return self.key == other.key
        return self.key == other

    def __gt__(self, other):
        if isinstance(other, self.__class__):
            return self.key > other.key
        return self.key > other

    def __ge__(self, other):
        if isinstance(other, self.__class__):
            return self.key >= other.key
        return self.key >= other

    def __lt__(self, other):
        if isinstance(other, self.__class__):
            return self.key < other.key
        return self.key < other

    def __le__(self, other):
        if isinstance(other, self.__class__):
            return self.key <= other.key
        return self.key <= other

## Override the Initializer

Next, we'll declare a new class called `KVStore` which will be a new extension of the `BTree` class above.

In [15]:
class KVStore(BTree):
    
    def __init__(self):
        super().__init__(split_threshold = 2)
        
    def add(self, key, value):
        node = self._find_node(self.root, key)
        if node is None:
            super().add(key, value)
        else:
            for i, node_key in enumerate(node.keys):
                if node_key == key:
                    node.values[i] = value

## Testing KVStore

We're going to test the implementation we just created to make sure KVStore works properly. To do this, we're going to add assertions that will ensure the state of the object is what it's supposed to be. 

In [16]:
kvs = KVStore()

# Test the split threshold
assert kvs.split_threshold == 2, "Split Threshold is 2"

# Test the .add() and .get_value() methods
for i in range(10):
    kvs.add(i, i)
    
for i in range(10):
    assert kvs.get_value(i) == i, "i is i"

# Testing two entries with the same key and different values
for i in range(10):
    kvs.add(i, i + 1)
    
for i in range(10):
    assert kvs.get_value(i) == i + 1, "i is i + 1"

TypeError: __init__() got an unexpected keyword argument 'split_threshold'

## Implement the Get & Set

Currently in progress...